In [1]:
import numpy as np
import pandas as pd
import os
import pickle

In [20]:
raw_data_dir = "../data/ft_data/"
new_dir = './BeFree/n_out/'
!cp ../data/ft_data/labels_n.txt ./BeFree/n_out/labels.txt
FT_MODE = True

In [2]:
# raw_data_dir = "../data/abs_data/2nd_ann/"
# new_dir = './BeFree/abs_out/'
# !cp ../data/abs_data/2nd_ann/labels.txt ./BeFree/abs_out/labels.txt
# FT_MODE = False

In [3]:
# generate gene/disease annotation for BeFree

In [8]:
sentence_path = os.path.join(raw_data_dir, "sentences.txt")
ner_path = os.path.join(raw_data_dir, "anns.txt")
text_path = os.path.join(raw_data_dir, "docs.txt")



new_gene_file = open(new_dir+"genes_FINAL.befree", 'w')
new_disease_file = open(new_dir+"diseases_FINAL.befree", 'w')


text_file = open(text_path, "r")
sentence_file = open(sentence_path, "r")
ner_file = open(ner_path, "r")

tar_pmid_lst = None

def get_sent_offset(sentences, text):
    sent_offset = []
    for i, sentence in enumerate(sentences):
        if i == 0:
            offset = 0
        else:
            offset = last_offset + len(last_sentence)
        if sentence != text[offset: offset+len(sentence)]:
            offset = offset + text[offset:].find(sentence)
        sent_offset.append(offset)        
        last_offset = offset
        last_sentence = sentence
    return sent_offset

def Sent_no(sent_offset, offset):
    return np.digitize(offset, sent_offset).tolist() - 1


def offset_in_sent(sentences_offset, sent_no, begin, end):
    begin_offset = begin - sentences_offset[sent_no]
    end_offset = end - sentences_offset[sent_no]
    return str(begin_offset) + "#" + str(end_offset)

def get_sentence(sentences, row):
    return sentences[sent_no]
    
    

_idx = 1
while (1):
    line = text_file.readline()
    if line == '':
        break
    pmid = line.strip()

    title = text_file.readline()
    title = title[:-1] if len(title) > 0 else title
    abstract = text_file.readline()
    abstract= abstract[:-1] if len(abstract) > 0 else abstract 
    para = text_file.readline()
    para = para[:-1] if len(para) > 0 else para 
    #fix space between title, abstract, para

    text = title + " " + abstract + " " + para

    text_file.readline()
    #text = abstract title + text

    sentences = []
    line = sentence_file.readline()
    if line == "\n":
        sentence_file.readline()
    while (1):
        line = sentence_file.readline()
        if line == "\n":
            break
        sentence = line.strip()
        sentences.append(sentence)

    sent_offset = get_sent_offset(sentences, text)

    anns = []
    while (1):
        line = ner_file.readline()
        if line == "\n":
            break
        ann = line.strip().split("\t")
        if ';' in ann[5]:
            ann[5] = ann[5].split(";")[1]
            
        # for ft only
        if FT_MODE:
            ann = ann[:-1]
#         print(ann)
        ann[1] = int(ann[1])
        ann[2] = int(ann[2])
        ann[-1], ann[-2] = ann[-2], ann[-1]
        if ann[4] == 'None' or len(ann[4]) < 1:
            continue
        anns.append(ann[:])
#         print(ann)
        begin, end, mention, _id, _type = ann[1], ann[2], ann[3], ann[4], ann[5]
        
        if tar_pmid_lst and (pmid not in tar_pmid_lst):
            continue
#         print(_type)
        sent_no = Sent_no(sent_offset, begin)
        off_tar = offset_in_sent(sent_offset, sent_no, begin, end)
#         print(sent_no, off_tar)
        
        
        _pmid = pmid
        _year, _journal_name, _journal_ISSN = '#', '#', '#'
        _section, _section_nb, _sent_no = 'TITLE', 0, 00.8
        
        if sent_no > 0:
            _section, _section_nb, _sent_no = 'ALL_TEXT', 1, sent_no

        _entity_id = _id
        _entity_type = 'LONGTERM|DICTIONARY'
        _entity_norm = mention.lower()
        _mention = mention
        _offset = off_tar
        _entity_parent = 'nan'
        _sentence = sentences[sent_no]
#     'pubmed', 'year','journal_name', 
# 'journal_ISSN', 'section', 'section_nb', 'sent_no', 'entity_id', 'entity_type', 'entity_norm', 'mention',  'offset',
# 'entity_parent',  'sentence'

    
#         print('%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t' % (\
#                     _pmid, _year, _journal_name, _journal_ISSN, \
#                     _section, _section_nb, _sent_no, \
#                     _entity_id,  _entity_type,  _entity_norm, \
#                     _mention, _offset, _entity_parent,\
#                     _sentence))
#         print(_type)
        if _type == 'Disease':
            new_disease_file.write('%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\n' % (\
                    _pmid, _year, _journal_name, _journal_ISSN, \
                    _section, _section_nb, _sent_no, \
                    _entity_id,  _entity_type,  _entity_norm, \
                    _mention, _offset, _entity_parent,\
                    _sentence))
        else:
            new_gene_file.write('%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\n' % (\
                    _pmid, _year, _journal_name, _journal_ISSN, \
                    _section, _section_nb, _sent_no, \
                    _entity_id,  _entity_type,  _entity_norm, \
                    _mention, _offset, _entity_parent,\
                    _sentence))
            
    if tar_pmid_lst and (pmid not in tar_pmid_lst):
#     if pmid in tar_pmid_lst:
        print(_idx, pmid)
        _idx += 1
#     print(abstract)
#     print(para)
    
#     print(sentences)
#     print(sent_offset)
    


#     print('\n'.join([str(i) for i in anns]))
#     break
    
print('end')
    
    
new_gene_file.close()
new_disease_file.close()
    

    
text_file.close()
sentence_file.close()
ner_file.close()

end


In [18]:
# run BeFree